In [ ]:
from PIL import Image
import numpy as np

In [ ]:
def getblocks(image: np.ndarray, blockshape: tuple, moveAxis: bool = True) -> np.ndarray:
    '''
    takes the array of image in grey= 2D and in RGB = 3D
    takes the numpy array and converts it the the blocks in the fastest way
    '''
    if len(image.shape) == 2:
        mode = "grey"
        image = image.reshape(*image.shape, 1)
    else:
        mode = "color"

    img_height, img_width, channels = image.shape
    tile_height, tile_width = blockshape

    if img_height % tile_height != 0 or img_width % tile_width != 0:
        print("warning: Block size is not fit for the image!")

    tiled_array = image.reshape(
        img_height//tile_height, tile_height, img_width//tile_width, tile_width, channels)
    tiled_array = tiled_array.swapaxes(1, 2)

    if moveAxis:
        tiled_array = tiled_array.reshape(-1,
                                          *(tile_height, tile_width, channels))
        tiled_array = np.moveaxis(tiled_array, source=len(
            tiled_array.shape)-1, destination=1)

    return tiled_array


def combineBlocks(tiled_array: np.ndarray, imageshape: tuple, blockshape: tuple, movedAxis: bool = True) -> np.ndarray:

    if len(imageshape) == 2:
        mode = "grey"
        imageshape = *imageshape, 1
    else:
        mode = "color"

    img_height, img_width, channels = imageshape
    tile_height, tile_width = blockshape

    if movedAxis:
        image = tiled_array.copy()
        image = image.reshape(img_height//tile_height, tile_height,
                              img_width//tile_width, tile_width, channels)
        swapaxisShape = list(image.shape)
        swapaxisShape[1], swapaxisShape[2] = swapaxisShape[2], swapaxisShape[1]
        image = image.reshape(swapaxisShape)
        image = image.swapaxes(1, 2)
    else:
        image = tiled_array
        # IDK have completed this else case. Btw we aren't using this case lol :)

    return image.reshape(imageshape)


In [ ]:
image = np.arange(1,(16*16)+1).reshape(16,16)
image

In [ ]:
hehe = getblocks(image, (4, 4))
hehe.shape, hehe.strides, hehe


In [ ]:
sixtenb=hehe.copy()
sixtenb.shape,sixtenb.strides, sixtenb

In [ ]:
lul = getblocks(sixtenb[0][0], (2, 2), False)
lul.shape, lul.strides, lul


In [ ]:
for i in range(16):
    sixtenb[i][:] = getblocks(sixtenb[i][0], (2, 2))
sixtenb.shape,sixtenb.strides, sixtenb

In [ ]:
hehe[0], hehe[100], hehe[6554]

In [ ]:
ret = combineBlocks(hehe, (512, 512), (16, 4))
ret.shape, ret.strides, ret

In [ ]:
(ret.reshape(-1) == image.reshape(-1)).all()


In [ ]:
for i in range(512):
    for j in range(512):
        if ret[i][j][0] != image[i][j]:
            print(i, j, ret[i][j][0], image[i][j])
